In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!pip install spacy-transformers


In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import warnings
warnings.filterwarnings("ignore")
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm
import spacy_transformers


In [5]:
df_train=pd.read_csv('final_train.csv', encoding='utf-8', encoding_errors='ignore')
df_train.columns=['sentence','tags']

df_test=pd.read_csv('final_test.csv', encoding='utf-8', encoding_errors='ignore')
df_test.columns=['sentence','tags']

In [6]:
# function to obtain entity format according to Spacy
def get_entity_tuple(df):
  list_of_entities = []
  for x in range(len(df)):
    d = {"entities": []}
    i, j = 0, 0
    idx = -1
    
    tokens = df["tags"][x].split()
    words = df['sentence'][x].split()
    if len(words) != len(tokens):
      continue

    temp = df["sentence"][x] + " "
    for char in temp:
      if char == " ":
        d["entities"].append((i, j, tokens[idx]))
        j += 1
        i = j
        idx += 1
      else:
        j += 1
    list_of_entities.append((df["sentence"][x], d))
  
  return list_of_entities

In [7]:
# apply on train and test set
train_entities = get_entity_tuple(df_train)
test_entities = get_entity_tuple(df_test)

In [ ]:
db = DocBin()
nlp = spacy.load("en_core_web_trf")

for text, annot in tqdm(train_entities): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./train.spacy") # save the docbin object


In [ ]:
db = DocBin()
nlp = spacy.load("en_core_web_trf")

for text, annot in tqdm(test_entities): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./test.spacy") # save the docbin object

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!python -m spacy train config_trf.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy --gpu-id 0